![Pytorch](images/pytorch_logo.png)
# Regression in Pytorch with all the bells and whistles
Pytorch lets us skip a bunch of math, but that's only part of why it's heavily used in Deep Learning. It is after all a Deep Learning framework, not a maths framework!

Let's redo our simple Linear Regression, but this time we use all the bells and whistles!

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
# Set seed
seed = 42
torch.manual_seed(seed);

## Load data

We are using the built-in sklearn dataset Boston House Prices.

Our goal is to predict the median price of a home in a given town from a number of features, such as Crime Rate, Property Tax Rate, amount of Industry etc.

It's generally a good idea to scale our data, so we use Sklearn's MinMax scaler to scale our values between 0 and 1

In [3]:
# Load our dataset
boston = load_boston()
train_x, test_x, train_y, test_y = train_test_split(boston.data, boston.target, random_state=seed)
scaler = MinMaxScaler()


train_x = torch.tensor(scaler.fit_transform(train_x), dtype=torch.float)
test_x = torch.tensor(scaler.transform(test_x), dtype=torch.float)
train_y = torch.tensor(train_y, dtype=torch.float).view(-1, 1)
test_y = torch.tensor(test_y, dtype=torch.float).view(-1, 1)

## Setup parameters

We have some hyperparameters to set, as well as some numbers we need to know upfront.

`layer_size` --> We need to know how many input variables there are, so we can create an equivalent number of weights

`lr` --> Aka learning rate.
When we take a step in our gradient descent, we multiply by this factor, so we don't take too big or too large a step. 

`epochs` --> How many times should we keep stepping?

In [4]:
# Set some parameters
layer_size = train_x.shape[1]
lr = 0.05
epochs = 700

# Defining the Model
We get a new toy here:

- `nn.Linear`

The linear class basically implements all that stuff we were doing by hand before, of setting up and keeping track of the `w` and `b` variables. This includes some initialization best practices, so we get that for free!

# Define loss function and optimizer
Pytorch predefines a number of loss functions for me, so I can just use those directly.
Another new features is the SGD optimizer. Until now we have been updating the weights "by hand" in a fairly naive fashion. There are many ways of updating our weights and Pytorch provides implementations for most of these. SGD is the closest to what we've been doing so far, so we use that to handle our weight updates.

In [15]:
model = nn.Linear(layer_size, 1) # Defines our parameters for us
loss_func = nn.MSELoss() # Define loss func
opt = optim.SGD(model.parameters(), lr=lr) # Tell the optimizer what parameters to keep track of

In [16]:
# Training loop
for epoch in range(epochs):
    # Forward pass
    pred = model(train_x)
    loss = loss_func(pred, train_y)
    
    # Backpropagation
    loss.backward() # The magic bit
    opt.step() # A new magic bit
    opt.zero_grad() # Gotta reset the gradients to zero, so they don't accumulate
    
    # Validate model
    with torch.no_grad():
        val_pred = model(test_x)
        val_loss = loss_func(val_pred, test_y) # Calculate validation loss
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} Training Loss: {loss.item()} Test Loss: {val_loss.item()}")

Epoch: 0 Training Loss: 624.5175170898438 Test Loss: 260.6249694824219
Epoch: 10 Training Loss: 96.25338745117188 Test Loss: 75.32975769042969
Epoch: 20 Training Loss: 75.75519561767578 Test Loss: 58.45773696899414
Epoch: 30 Training Loss: 64.2809066772461 Test Loss: 49.66794204711914
Epoch: 40 Training Loss: 57.41917037963867 Test Loss: 44.96377944946289
Epoch: 50 Training Loss: 52.97154235839844 Test Loss: 42.2337646484375
Epoch: 60 Training Loss: 49.82622528076172 Test Loss: 40.45537567138672
Epoch: 70 Training Loss: 47.4166145324707 Test Loss: 39.140419006347656
Epoch: 80 Training Loss: 45.45014572143555 Test Loss: 38.05980682373047
Epoch: 90 Training Loss: 43.772483825683594 Test Loss: 37.108489990234375
Epoch: 100 Training Loss: 42.29939270019531 Test Loss: 36.23963165283203
Epoch: 110 Training Loss: 40.9825325012207 Test Loss: 35.432945251464844
Epoch: 120 Training Loss: 39.792274475097656 Test Loss: 34.67971420288086
Epoch: 130 Training Loss: 38.70893859863281 Test Loss: 33.975